### **Paso 7.1 - Crear un Pipeline de orquestación de los notebooks de ingestación**

#### 1. Agregar actividad **Databricks Notebook** para la ingestación del archivo **circuits.csv**

Creamos el pipeline que lleva por nombre **pl_ingest_formula1_data** y agregamos la primer actividad **Databricks Notebook** que lleva por nombre **Ingest Circuits File** y que ejecutará el notebook donde realizamos la ingestación del archivo desde la ruta **/mnt/formula1dl/raw/{v_file_date}/circuits.csv** hacia una tabla delta, es decir, se creará una tabla delta llamada **circuits** en la base de datos **f1_processed** y se almacenara la data en la ruta  **/mnt/formula1dl/processed/circuits** del data lake.

<center><img src="https://i.postimg.cc/L4ffwXDp/db193.png"></center>

Creamos el Linked Service para conectarnos con Databricks

<center><img src="https://i.postimg.cc/Dz84gy0R/db194.png"></center>

Pero previo a ello, dado que utilizaremos **Managed indentity** en el tipo de autenticación al momento de crear el Linked Service, debemos generar un rol de acceso para la aplicación de Azure Data Factory (**databricks-course-adf**) en el workspace de Databricks

<center><img src="https://i.postimg.cc/bNhbhhV2/db196.png"></center>
<center><img src="https://i.postimg.cc/x1sbMbfK/db197.png"></center>

Ahora creamos el Linked Service que haga referencia hacia Databricks

<center><img src="https://i.postimg.cc/RVLtCD2p/db195.png"></center>
<center><img src="https://i.postimg.cc/1XZgsGBZ/db198.png"></center>
<center><img src="https://i.postimg.cc/vZ8V4vsY/db199.png"></center>

Debemos seleccionar la ruta donde se encuentra el notebook de ingestación del archivo **circuits.csv**

<center><img src="https://i.postimg.cc/3NCkRkf0/db202.png"></center>
<center><img src="https://i.postimg.cc/d1C7bH8B/db200.png"></center>

Dado que nuestros notebooks de ingestación poseen 2 parámetros: fecha (**p_file_date**)  y source (**p_data_source**), debemos crearlos tanto a nivel de actividad como a nivel de pipeline
```
A nivel de pipeline se crea la variable:

v_data_source = Eargast
```
<center><img src="https://i.postimg.cc/c1QgTz11/db203.png"></center>

Y creamos también
```
A nivel de pipeline el parámetro sin valor aún:

p_window_end_date = 
```
<center><img src="https://i.postimg.cc/MKqcLYX7/db204.png"></center>

Y los parámetros del notebook **Ingest Circuits file** heredarán los valores entregados tanto a la variable **v_data_source** como al parámetro **p_window_end_date**
```
p_data_source = @variables('v_data_source')
p_file_date = @formatDateTime(pipeline().parameters.p_window_end_date,'yy-MM-dd')
```
<center><img src="https://i.postimg.cc/s2qQ0dZx/db201.png"></center>

#### 2. Agregar actividad **Databricks Notebook** para cada ingestación de archivos restantes**

Nos queda por agregar una actividad **Databricks Notebook** para:

    -   races.csv
    -   constructors.json
    -   drivers.json
    -   results.json
    -   pit_stops.json
    -   lap_times
    -   qualifying

Para ello, copiaremos la actividad **Ingest Circuits files** y cambiaremos el nombre de la actividad y la ruta de la ubicación de archivo para cada uno

<center><img src="https://i.postimg.cc/KvxGBd7n/db205.png"></center>

Si ejecutamos los notebooks de la siguiente manera, todos se ejecutaran al mismo tiempo. Si necesitamos que se ejecuten después de que termine de ejecutarse uno previamente, debemos conectarlos por medio de la flecha

<center><img src="https://i.postimg.cc/Pryt42K0/db206.png"></center>
<center><img src="https://i.postimg.cc/4x3XFVSy/db207.png"></center>

#### 3. Agregar actividad **Get Metadata** y **If Conditional** para condicionar ejecuciones 


Vamos a utilizar las actividades **Get Metadata** y **If Conditional** para condicionar las ejecuciones de los notebooks. Esto por el hecho de ejecutar los notebooks solo si la fecha pasada al parámetro a nivel de pipeline **p_file_date** corresponde a algun directorio que se encuentre en el contenedor **raw**

Vamos a empezar agregando la actividad **Get Metadata**

<center><img src="https://i.postimg.cc/ZnPPMCqY/db208.png"></center>

Esta actividad nos va a pedir crear un nuevo Dataset y Linked Service

<center><img src="https://i.postimg.cc/nVq1zw18/db209.png"></center>

El Dataset hará referencia hacia Azure Data Lake Storage

<center><img src="https://i.postimg.cc/mkSMSJdt/db210.png"></center>

Dado que tenemos archivos del tipo CSV y JSON, en esta instancia en realidad no importa el tipo de archivo que seleccionemos

<center><img src="https://i.postimg.cc/N0W1G19f/db211.png"></center>

Nuestro dataset llevará por nombre **ds_formula1_raw**. Como mencionamos, debemos crear un Linked Service que también haga referencia hacia Azure Data Lake Storage

<center><img src="https://i.postimg.cc/CLYsfHvR/db212.png"></center>
<center><img src="https://i.postimg.cc/Bb3HH4Br/db213.png"></center>

Despues de crear el Linked Service, debemos terminar de configurar nuestro Dataset, al cual le crearemos el parametro **p_window_end_date** (a nivel de dataset)

<center><img src="https://i.postimg.cc/zfLC1dfc/db214.png"></center>
<center><img src="https://i.postimg.cc/J4qc3DgP/db215.png"></center>

Vamos a utilizar el parámetro (formateado) que creamos a nivel de dataset, como nombre del directorio
```
@formatDateTime(dataset().p_window_end_date,'yy-MM-dd')
```
<center><img src="https://i.postimg.cc/9QgdK3c9/db216.png"></center>

Vamos a pasarle al parámetro a nivel de dataset **p_window_end_date** el valor que le indiquemos al parámetro **p_window_end_date** que creamos a nivel de pipeline al momento de ejecutar el pipeline
```
p_window_end_date = pipeline().parameters.p_window_end_date
```
Y además en **Field list** indicamos la opción de **Exists**, que almacenará el valor (**True** o **False**) que tome el parámetro a nivel de dataset **p_window_end_date** (que tomará el valor del parámetro **p_window_end_date** que creamos a nivel de pipeline y que ingresaremos al momento de ejecutar el pipeline) existe o no existe

<center><img src="https://i.postimg.cc/nrn9MTHZ/db217.png"></center>

Agregamos una actividad **If Conditional** que evaluará el valor que devuelva la opción **Exists** de la actividad **Get Metadata** y de acuerdo a ese valor ejecutará una tarea si este valor es verdadero o es falso

<center><img src="https://i.postimg.cc/RhG6tfFt/db218.png"></center>

En la pestaña **Activities** en la opción de **Expressions** indicamos la expresión que capturá el valor obtenido por **Exists** de la actividad **Get Metadata**
```
@activity('Get Folder Details').output.Exists
```
<center><img src="https://i.postimg.cc/7YsJYgj8/db219.png"></center>

Luego, tomamos todas las actividades **Databricks notebook** las cortamos y las pegamos dentro de la actividad **True**. Esto vale decir que al momento de evaluarse lo devuelto por la opción **Exists** de la actividad **Get Metadata** es igual a **True** se ejecutarán las actividades **Databricks notebook**. Si es **False** no pasará nada, pues no agregaremos ninguna actividad

<center><img src="https://i.postimg.cc/QxxW364J/db220.png"></center>
<center><img src="https://i.postimg.cc/Zq3WcXn4/db221.png"></center>
<center><img src="https://i.postimg.cc/x1XJkXSk/db222.png"></center>

Validaremos y ejecutaremos el pipeline utilizando **Debug**

<center><img src="https://i.postimg.cc/yx6W7dP8/db223.png"></center>

E ingresaremos el valor **2021-03-20**. Este valor de directorio no existe en el contenedor **raw** en ADLS. Por tanto, **Exists** devolverá **False** y no se ejcutará nada. 

<center><img src="https://i.postimg.cc/4dQmCtp0/db224.png"></center>
<center><img src="https://i.postimg.cc/nzJCwT97/db225.png"></center>
<center><img src="https://i.postimg.cc/RhPhhBsQ/db226.png"></center>

Ahora ingresaremos el valor **2021-03-21**. Este valor de directorio si existe en el contenedor **raw** en ADLS. Por tanto, **Exists** devolverá **True** y se ejcutarán as actividades **Databricks notebook**

<center><img src="https://i.postimg.cc/BQ6tyjbh/db227.png"></center>
<center><img src="https://i.postimg.cc/J7KGPNrD/db228.png"></center>
<center><img src="https://i.postimg.cc/c4b6Nd8d/db229.png"></center>